In [ ]:
import os
import numpy as np
import pandas as pd

from calculation import AngleCalculation, VerticalCalculation

In [ ]:
#Using percentage as interval
def interval_test(sample, controlled, CI, SAl, SAl_min=5):
    min_CI = controlled * (1-CI)
    max_CI = controlled * (1+CI)
    result = (sample >= min_CI) & (sample <= max_CI) | (SAl < SAl_min)
    return result

#Using percentage as interval
def max_test(sample, controlled, max, SAl, SAl_min=5):
    max_CI = controlled * (1+max)
    result = (sample <= max_CI) | (SAl < SAl_min)
    return result

#Using number as interval
def interval_test_vertical(sample, controlled, CI):
    min_CI = controlled - CI
    max_CI = controlled + CI
    result = (sample >= min_CI) & (sample <= max_CI) | (sample == 0)
    # print(result)
    return result

def test1(df):
    ETR = 1367
    test1_1 = (df['Eeg']>0) & (df['Eeg']< 1.2*ETR) | (df['SAl'] < 5)
    test1_2 = (df['Eed']>0) & (df['Eed']< 0.8*ETR) | (df['SAl'] < 5)
    test1_3 = (df['Eeb']>=0) & (df['Eeb']< 1.0*ETR) | (df['SAl'] < 5)
    test1_passed = test1_1 & test1_2 & test1_3
    return test1_passed

def test2(df):
    EegCalCorrect = df['Eed'] + \
        (1.15 * df['Eeb'] * np.sin(df['SAl']*np.pi/180))
    test2_1 = interval_test(df['Eeg'], EegCalCorrect,
                            0.25, df['SAl'])
    test2_3 = max_test(df['Eed'], df["Eeg"], 0.1, df['SAl'])
    test2_passed = test2_1 & test2_3
    return test2_passed

def test3(df):
    test3_1 = {}
    for d in ['N', 'E', 'S', 'W']:
        test3_1[d] = interval_test_vertical(df[f'Ee{d}'], df[f'Eeng{d}'], 250)
        # test3_1[d] = interval_test(df[f'Ee{d}'], df[f'Eeng{d}'],0.20, df['SAl'])
    # test3_passed = test3_1['N'] & test3_1['E'] & test3_1['S'] & test3_1['W']
    return test3_1['N'], test3_1['E'], test3_1['S'], test3_1['W']

def test4(df):
    temperature_test = (df['Temperature'] >= 5) & (df['Temperature'] <= 60)
    humidity_test = (df['RH'] >= 0) & (df['RH'] <= 100)
    test4_passed = temperature_test & humidity_test
    return test4_passed
    
def diffuse_test(df):
    diffuse_test_passsed = df['Eed'] <= 650
    return diffuse_test_passsed


In [ ]:
files = os.listdir("correcteddata")
files.sort()

for file in files:
    df_correcteddata = pd.read_csv(f"correcteddata/{file}", header=0, parse_dates=['timestamp'], index_col=0)

    df_withangle = AngleCalculation(df_correcteddata)

    df_test_ETR = test1(df_withangle)
    df_test_horizon = test2(df_withangle)
    df_vertical = VerticalCalculation(df_withangle)
    df_test_N, df_test_E, df_test_S, df_test_W = test3(df_vertical)
    df_test_temphumid = test4(df_withangle)
    df_test_diffuse = diffuse_test(df_withangle)

    df_with_valid = pd.DataFrame([], index=df_withangle.index)
    df_with_valid["ETR"] = df_test_ETR.values
    df_with_valid["horizon"] = df_test_horizon.values
    df_with_valid["N"] = df_test_N.values
    df_with_valid["E"] = df_test_E.values
    df_with_valid["S"] = df_test_S.values
    df_with_valid["W"] = df_test_W.values
    df_with_valid["temphumid"] = df_test_temphumid.values
    df_with_valid["diffuse"] = df_test_diffuse.values
    df_with_valid = df_with_valid.replace(True, "Passed").replace(False, "Failed")
    df_with_valid.to_csv(f"aqc/{file}", index=True, header=True)